In [16]:
import datetime
from datetime import datetime, timedelta
import calendar
import requests
import json
import urllib3
import configparser
import csv
config = configparser.ConfigParser()
config.read('credentials.ini')
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

MDM_add = config['API']['MDM_add']
MDM_user = config['API']['MDM_username']
MDM_password = config['API']['MDM_password']
HES_add = config['API']['HES_add']
HES_user = config['API']['HES_username']
HES_password = config['API']['HES_password']

print("***** Where do you want to get data from HES or MDM *****")
while True:
    data_source = input("Enter the data source (HES or MDM): ").strip()

    if data_source == "HES":
        add = HES_add
        user = HES_user
        password = HES_password
        break
    elif data_source == "MDM":
        add = MDM_add
        user = MDM_user
        password = MDM_password
        break
    else:
        print("Invalid choice. Please enter either 'HES' or 'MDM'.")
        


print(f"Would you like to export data for all devices in {data_source} or only for devices listed in the CSV file?")
while True:
    print(f"Press 1 to export data for all devices in {data_source}.")
    print(f"Press 2 to export data for devices from the CSV file {data_source}.")
    dev_type = input("Input 1 or 2 and press Enter: ")
    if dev_type == '1':
        break
    elif dev_type == '2':
        dev_file_path = input("Enter the file name like 'device_list.csv': ")
        break
    else:
        print("Invalid choice. Please enter either '1' or '2'.")


def csv_to_device(filepath):
    dev_list = []
    try:
        with open(filepath, mode='r', encoding='utf-8-sig') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                if row:
                    dev_list.append(list(row.values())[0])
    except FileNotFoundError:
        print(f"Error: The file '{filepath}' was not found.")
    except IOError:
        print(f"Error: An I/O error occurred while reading the file '{filepath}'.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return dev_list


def billing_time_data(month: str = None, year: int = None, start_hour: int = 18, start_minute: int = 29):
    # Use current year and month if not provided
    now = datetime.now()
    if year is None:
        year = now.year
    if month is None:
        month = now.strftime('%B')
    
    month = month.capitalize()
    cur_start_date = datetime(year, list(calendar.month_name).index(month), 
                              calendar.monthrange(year, list(calendar.month_name).index(month))[1],
                              start_hour, start_minute)
    start_time_cur = cur_start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_cur = (cur_start_date + timedelta(minutes=2)).strftime("%Y-%m-%dT%H:%M:%SZ")        
    return start_time_cur, end_time_cur


def daily_time_data(day: int = None, month: str = None, year: int = None, start_hour: int = 18, start_minute: int = 29):
    now = datetime.now()    
    if year is None:
        year = now.year
    if month is None:
        month = now.strftime('%B')
    month = month.capitalize()    
    if day is None:
        day = now.day
        month = now.strftime('%B')
        year = now.year
    else:
        date_obj = datetime(year, list(calendar.month_name).index(month), day) - timedelta(days=1)
        year, month, day = date_obj.year, date_obj.strftime('%B'), date_obj.day
    cur_start_date = datetime(year, list(calendar.month_name).index(month), day, start_hour, start_minute)
    start_time_cur = cur_start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_time_cur = (cur_start_date + timedelta(minutes=2)).strftime("%Y-%m-%dT%H:%M:%SZ")

    return start_time_cur, end_time_cur


#Take user input for profile, year, month, date depending on the type of profile users wants to extract data for
while True:
    profile = input("Enter the profile you want to extract data for (Billing or Daily): ").strip().lower()
    if profile == "billing":
        prof = "1-0:98.1.0*255"
        tprof = "Billing Profile"
        while True:
            try:
                month_input = input("Enter the month (e.g., January) or press Enter to use the current month: ")
                year_input = input("Enter the year (e.g., 2024) or press Enter to use the current year: ")
                month = month_input if month_input else None
                year = int(year_input) if year_input else None
                start_time_cur, to_time_cur = billing_time_data(month, year)
                break
            except (ValueError, IndexError):
                print("Invalid date. Please try again.")
        break
    elif profile == "daily":
        prof = "1-0:99.2.0*255"
        tprof = "Daily Profile"
        while True:
            try:
                year_input = input("Enter the year (e.g., 2024) or press Enter to use the current year: ")
                month_input = input("Enter the month (e.g., January) or press Enter to use the current month: ")
                day_input = input("Enter the day (e.g., 31) or press Enter to use today's date: ")
                month = month_input if month_input else None
                year = int(year_input) if year_input else None
                day = int(day_input) if day_input else None
                start_time_cur, to_time_cur = daily_time_data(day, month, year)
                break
            except (ValueError, IndexError):
                print("Invalid Date. Please try again")
        break
            
    else:
        print("Invalid choice. Please enter either 'billing' or 'daily'.")
        
end_date_part = start_time_cur.split("T")[0]

print("Data Will Be Extracted For Following: ")
print("======================================")
print(f"Data Source   : {data_source}")
print(f"Profile       : {tprof}")
print(f"Profile Value : {prof}")
print(f"Start Time    : {start_time_cur}")
print(f"End Time      : {to_time_cur}")
if dev_type == '1':
    print(f"Devices      : All Devices in {data_source}")
else:
    print(f"Devices      : All Devices in the {dev_file_path} file")


***** Where do you want to get data from HES or MDM *****


Enter the data source (HES or MDM):  MDM


Would you like to export data for all devices in MDM or only for devices listed in the CSV file?
Press 1 to export data for all devices in MDM.
Press 2 to export data for devices from the CSV file MDM.


Input 1 or 2 and press Enter:  1
Enter the profile you want to extract data for (Billing or Daily):  Billing
Enter the month (e.g., January) or press Enter to use the current month:  July
Enter the year (e.g., 2024) or press Enter to use the current year:  


Data Will Be Extracted For Following: 
Data Source   : MDM
Profile       : Billing Profile
Profile Value : 1-0:98.1.0*255
Start Time    : 2024-07-31T18:29:00Z
End Time      : 2024-07-31T18:31:00Z
Devices      : All Devices in MDM


In [17]:
def get_devices(add, user, password):
    url= f"{add}/api/1/devices/"
    try:
        r = requests.get(url, auth=(MDM_user, MDM_password), verify=False)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        return []
    
    device = r.json()
    
    del_key = ['communicationId', 'typeId', 'typeName', 'templateId', 'templateName', 'managementState', 'description', 'manufacturer', 'model', 'parentId', 'location', 'storeData', 'groupId']
    for item in device:
        for k in del_key:
            item.pop(k, None)
    return device

In [18]:
dev_list=get_devices(MDM_add, MDM_user, MDM_password)

In [20]:
print(dev_list[0])

{'id': 'ISKIE077058', 'inventoryState': 'installed', 'lastConnection': 1729567860, 'groupName': 'Panchet Commercial'}


In [21]:
schema = [
    {"device": devices.get('id'), "profile": prof, "from": start_time_cur, "to": to_time_cur}
    for devices in dev_list
]
def split_schema(biglist, shard_size):
    # Using list comprehension to create sublists of the given size
    return [biglist[i:i + shard_size] for i in range(0, len(biglist), shard_size)]
sp_schema = split_schema(schema, 3000)

In [22]:
def get_billing_profile(add, username, password, schema):
    m_values_kwh = []
    m_values_kvah = []
    request_url = f"{add}/api/1/bulk/device-profiles/metered-data/get"

    try:
        r = requests.post(request_url, auth=(username, password), verify=False, json=schema)
        r.raise_for_status()  # Raise an error for bad status codes (4xx, 5xx)
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # For HTTP errors
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred: {conn_err}")  # For connection errors
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred: {timeout_err}")  # For timeout errors
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")  # For all other request errors
    else:
        try:
            val = r.json()  # Try to parse the response as JSON
        except ValueError as json_err:
            print(f"JSON decoding error: {json_err}")
        else:
            # Process the JSON data here
            print("Request successful, and JSON parsed.")
            return val
    

In [35]:
def process_metered_values(json_data):
    m_values_kwh = []
    m_values_kvah = []
    entries = json_data.get("value", [])
    if entries:
        first_entry = entries[0]
        capturedAt = first_entry.get("capturedAt", "")
        metered_values = first_entry.get("meteredValues", [])
        for item in metered_values:
            if item["registerId"] == "1-0:1.8.0*255":
                if not item.get("measuredAt"):
                    item["measuredAt"] = capturedAt
                m_values_kwh.append(item)
            
            elif item["registerId"] == "1-0:9.8.0*255":
                if not item.get("measuredAt"):
                    item["measuredAt"] = capturedAt
                m_values_kvah.append(item)
    
    return m_values_kwh, m_values_kvah


In [58]:
val = get_billing_profile(MDM_add, MDM_user, MDM_password, sp_schema[0])

Request successful, and JSON parsed.


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [47]:
def process_metered_values(json_data):
    m_values_kwh = []
    m_values_kvah = []

    for record in json_data: 
        if record.get("success"):
            entries = record.get("value", [])

            if entries:
                first_entry = entries[0]
                capturedAt = first_entry.get("capturedAt", "")
                metered_values = first_entry.get("meteredValues", [])

                for item in metered_values:
                    if item["registerId"] == "1-0:1.8.0*255":
                        if not item.get("measuredAt"):
                            item["measuredAt"] = capturedAt
                        m_values_kwh.append(item)
                    elif item["registerId"] == "1-0:9.8.0*255":
                        if not item.get("measuredAt"):
                            item["measuredAt"] = capturedAt
                        m_values_kvah.append(item)

    return m_values_kwh, m_values_kvah


In [91]:
def flatten_dicts(nested_list):
    flat_list = []
    
    for item in nested_list:
        if 'value' in item:
            for value in item['value']:
                # Prepare a flat dictionary starting with the current item
                flat_item = {
                    'success': item.get('success'),
                    'capturedAt': value.get('capturedAt'),
                    'readingReason': value.get('readingReason'),
                    'statusWord': value.get('statusWord'),
                    'dataSource': value.get('dataSource'),
                }
                
                # Flatten meteredValues if present
                if 'meteredValues' in value:
                    for meter in value['meteredValues']:
                        flat_meter = flat_item.copy()  # Copy base flat_item
                        flat_meter.update({
                            'registerId': meter.get('registerId'),
                            'value': meter.get('value'),
                            'unit': meter.get('unit'),
                            'measuredAt': meter.get('measuredAt'),
                        })
                        flat_list.append(flat_meter)
                else:
                    flat_list.append(flat_item)
    
    return flat_list

# Example usage
flattened_list = flatten_dicts(val)


In [94]:
flat_list=flatten_dicts(val)
print(flat_list[0])

{'success': True, 'capturedAt': '2024-07-31T18:30:00Z', 'readingReason': 1, 'statusWord': 0, 'dataSource': 64, 'registerId': '0-0:0.1.2*255', 'value': 1722450600, 'unit': 's', 'measuredAt': '2024-07-31T18:30:00Z'}


In [50]:
def process_metered_values(json_data):
    m_values_kwh = []
    m_values_kvah = []

    for record in json_data:  # Loop through each dictionary in the list
        if record.get("success"):
            entries = record.get("value", [])

            if entries:
                first_entry = entries[0]
                capturedAt = first_entry.get("capturedAt", "")
                metered_values = first_entry.get("meteredValues", [])

                for item in metered_values:
                    if item["registerId"] == "1-0:1.8.0*255":
                        if not item.get("measuredAt"):
                            item["measuredAt"] = capturedAt
                        m_values_kwh.append(item)
                    elif item["registerId"] == "1-0:9.8.0*255":
                        if not item.get("measuredAt"):
                            item["measuredAt"] = capturedAt
                        m_values_kvah.append(item)
        else:
            print(f"Error in record: {record.get('errorMessage', 'No error message provided')}")
            continue 

    return m_values_kwh, m_values_kvah


{'registerId': '1-0:9.8.0*255', 'value': 615.22, 'unit': 'kVAh', 'measuredAt': '2024-07-31T18:30:00Z'}
